In [1]:
from TST import *
from loss import MaskedMSELoss
import torch

In [3]:
def make_model(src, mask, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    feat_dim = src.shape[1]  # dimensionality of data features
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    pe = PositionalEncoding(d_model, dropout)
    proj = nn.Linear(feat_dim, d_model)
    output = nn.Linear(d_model, feat_dim)
    model = output(Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N)(pe(proj(src)), mask))
    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        print("p ", p)
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [4]:
# Synthetic Data
src = torch.randn(10, 512, 512)
X_test = torch.randn(10, 512, 512)
print("src.shape: ", src.shape)
# Create model
print("Creating model...")
TS_model = make_model(src, mask= None)
# loss function
loss_fn = nn.MaskedMSELoss()
# Initialize optimizer
optimizer = torch.optim.Adam(TS_model.parameters(), lr=0.0001)

src.shape:  torch.Size([10, 512, 512])
Creating model...


In [ ]:
# train model
BATCH_SIZE = 128
for epoch in range (10000) :
    for start in range (0 , src[-1] , BATCH_SIZE) :
        end = start + BATCH_SIZE
        batch = src[start:end]
        batch_pred = TS_model(batch)
        loss = loss_fn(batch_pred , batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss_fn(TS_model(src) , src)
    if epoch%100 == 0:
        print(' epoch {} training loss {} '.format(epoch , loss.item()))
    if(epoch%1000 == 0) :
        Y_test_pred = TS_model(X_test)
        print(Y_test_pred)